In [ ]:
##########################################################

# Hier werden die vorverarbeiteten Beschwerde-Texte in numerische Vektoren umgewandelt.
# Dazu nutzen wir:
# 1. TF-IDF: Gewichtung relevanter Begriffe basierend auf Häufigkeit und Seltenheit
# 2. Bag-of-Words: Zählt die Häufigkeit von Wörtern in den Texten

##########################################################
# Die Ergebnisse werden wieder gespeichert um mit dem nächsten Notebook die Themen zu Extrahieren und Analysieren

import pandas as pd
import numpy as np
import os
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import gensim
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
import matplotlib.pyplot as plt
import seaborn as sns
import re
import joblib  # Sichere Alternative zu pickle für das Speichern von Modellen
import json    # Zum Speichern von Metadaten

In [ ]:
# Versuchen Daten einzulesen
try:
    processed_df = pd.read_csv('../data/processed_complaints_v2.csv')
    print("Datei gefunden.")
    print(f"Geladene Texte: {len(processed_df)}")
except FileNotFoundError:
    print("Fehler: Datei 'processed_complaints_v2.csv' nicht gefunden. Zuerst 1. Datenvorverarbeitung ausführen")

In [ ]:
# Korpus erstellen, NaN-Werte entfernen sowie Numpy Array zu Listen konvertieren
corpus = processed_df['processed_text'].dropna().tolist()

In [ ]:
##########################################################
# 1. TF-IDF Vektorisierung
##########################################################

tfidf_vectorizer = TfidfVectorizer(
    max_features = 2000, # Limitiert auf die 2000 relevantesten Begriffe für effiziente Berechnung und Fokus auf Hauptthemen
    min_df = 5,          # Begriffe müssen in mindestens 5 Dokumenten vorkommen, um Rauschen zu reduzieren
    max_df= 0.75,        # Ignoriert Begriffe, die in mehr als 75% der Dokumente vorkommen (zu allgemein)
    ngram_range=(1, 2)   # Berücksichtigt Einzelwörter und Wortpaare für kontextuelle Begriffe
)


In [ ]:
# Vektorisierung durchführen
tfidf_matrix = tfidf_vectorizer.fit_transform(corpus)
print(f"TF-IDF-Matrix erstellt mit Shape: {tfidf_matrix.shape}")
print("Beispielbegriffe:", list(tfidf_vectorizer.get_feature_names_out()[:10]))

In [ ]:
# Speichern des TF-IDF Modells
joblib.dump(tfidf_vectorizer, '../models/tfidf_vectorizer.joblib')
np.save('../models/tfidf_matrix.npy', tfidf_matrix.toarray())

# Hier kam es zu einem Fehler mit einem Datentyp int64 um das Vokabular als JSON zu speichern. 
# Deshalb wird der Datentyp hier noch geändert. 

vocabulary_clean = {k: int(v) for k, v in tfidf_vectorizer.vocabulary_.items()}

# Speichern des Vokabulars als JSON für bessere Interpretierbarkeit
vocabulary = tfidf_vectorizer.vocabulary_
with open('../models/tfidf_vocabulary.json', 'w', encoding='utf-8') as f:
    json.dump(vocabulary_clean, f, ensure_ascii=False, indent=4)

In [ ]:
# Mittelwert je Spalte (= Begriff) berechnen
tfidf_means = np.asarray(tfidf_matrix.mean(axis=0)).flatten()

# Begriffe holen
terms = tfidf_vectorizer.get_feature_names_out()

# Top 20 Begriffe mit höchstem durchschnittlichen TF-IDF-Wert
top_n = 20
top_indices = tfidf_means.argsort()[::-1][:top_n]
top_terms = [terms[i] for i in top_indices]
top_scores = [tfidf_means[i] for i in top_indices]

# Balkendiagramm erstellen
plt.figure(figsize=(12, 6))
plt.barh(top_terms[::-1], top_scores[::-1])  # Ohne [::-1] sind die niedrigsten Werte oben
plt.xlabel("Durchschnittlicher TF-IDF-Wert")
plt.title("Top 20 Begriffe nach durchschnittlichem TF-IDF")
plt.tight_layout()

In [ ]:
##########################################################
# 2. Bag-of-Words Vektorisierung für LDA
##########################################################
# https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html Erklärung
# Bag-of-Words Vektorisierung mit CountVectorizer

count_vectorizer = CountVectorizer(
    max_features=5000,
    min_df=5,
    max_df=0.75,
    ngram_range=(1, 2)
)

# Count Matrix berechnen
count_matrix = count_vectorizer.fit_transform(corpus)
print(f"Count Matrix erstellt: {count_matrix.shape[0]} Dokumente/Zeilen, {count_matrix.shape[1]} Features")

In [ ]:
# Speichern des Count Vectorizers (BoW) und der Matrix für die LDA-Analyse
joblib.dump(count_vectorizer, '../models/count_vectorizer_for_lda.joblib')
np.save('../models/count_matrix_for_lda.npy', count_matrix.toarray())

In [ ]:
# Speichern des Vokabulars (für die Interpretation der LDA-Themen)
count_vocabulary = count_vectorizer.vocabulary_

# Konvertiere alle numpy.int64-Werte zu normalen Python int, auch hier kam es zu einem Datentyp Fehler
count_vocabulary = {key: int(value) for key, value in count_vocabulary.items()}

# Speichern des Vokabulars als JSON
with open('../models/count_vocabulary.json', 'w', encoding='utf-8') as f:
    json.dump(count_vocabulary, f, ensure_ascii=False, indent=4)


In [ ]:
# Visualisierung der häufigsten Terme 
term_freq = np.sum(count_matrix.toarray(), axis=0)
count_terms = count_vectorizer.get_feature_names_out()

# Sortiere nach Häufigkeit und zeige die Top 15 Begriffe
term_freq_sorted = sorted(zip(count_terms, term_freq), key=lambda x: x[1], reverse=True)
top_terms = term_freq_sorted[:15][::-1] # Top 15 Begriffe umgekehrt sortiert

# Daten für das Balkendiagramm vorbereiten
plt.figure(figsize=(12, 8))
plt.barh([term[0] for term in top_terms], [term[1] for term in top_terms])
plt.xlabel('Absolute Häufigkeit')
plt.title('Top 15 häufigste Begriffe im Korpus')
plt.tight_layout()

